## Pull in UDFs

In [ ]:
%run nb_udfs

## Run Capacities

In [ ]:
workspace = 'Admin%20Center' #have to escape the & symbol and spaces
lakehouse = 'lh_monitoring'

capacities_table = 'dimCapacities'


In [ ]:
#get a list of all capacities and load to a table in the lakehouse
response = _base_api(
        request=f"/v1.0/myorg/admin/capacities",
        method="get"
    )
df_capacities = pd.json_normalize(response.json()['value'])
df_capacities = spark.createDataFrame(df_capacities)
df_capacities = df_capacities.drop("admins","users")
#df_capacities.printSchema()
#print(df_capacities.columns)

In [ ]:
#creates a slowly changing dimension so we can keep an eye on any deleted capacities
capacitiesPath = udf_GetFilePath(workspace, lakehouse, capacities_table)
print(capacitiesPath)
naturalKeyColumnList = ['id']
primaryKeyColumnName = "tableId"
returnVal = udf_UpsertDimension(df_capacities,2,capacitiesPath,naturalKeyColumnList,primaryKeyColumnName,True)
print(returnVal)